# Stuff Documents Chain 简单拼接

## 源码解析

### 类声明和装饰器

In [ ]:
@deprecated(
    since="0.2.13",
    removal="1.0",
    message=(
        "This class is deprecated. Use the `create_stuff_documents_chain` constructor "
        "instead. See migration guide here: "
        "https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain/"  # noqa: E501
    ),
)
class StuffDocumentsChain(BaseCombineDocumentsChain):

- `@deprecated` 装饰器标识该类将被弃用，并指引用户使用新的构造函数 `create_stuff_documents_chain`。
- 该类继承自 `BaseCombineDocumentsChain`，意味着它实现了基于文档组合的功能。

### 类属性

In [ ]:
llm_chain: LLMChain
document_prompt: BasePromptTemplate = Field(default_factory=lambda: DEFAULT_DOCUMENT_PROMPT)
document_variable_name: str
document_separator: str = "\n\n"

- `llm_chain`：用于处理组合后的文档和其他输入的 LLM 链。
- `document_prompt`：用来格式化每个文档的提示模板。默认使用 DEFAULT_DOCUMENT_PROMPT。
- `document_variable_name`：将格式化后的文档字符串放置在 `llm_chain` 中的变量名。
- `document_separator`：在格式化后的文档字符串之间使用的分隔符，默认为换行符 \n\n。

### 配置类

In [ ]:
class Config:
    arbitrary_types_allowed = True
    extra = "forbid"

- `arbitrary_types_allowed`：允许在 `Pydantic` 模型中使用任意类型。
- `extra = "forbid"`：禁止在模型中定义未声明的额外字段。

### `get_default_document_variable_name` 方法

In [ ]:
@root_validator(pre=True)
def get_default_document_variable_name(cls, values: Dict) -> Dict:

- `get_default_document_variable_name` 是一个根验证器，用于在实例化前设置或验证 `document_variable_name`。
- 该方法会检查 `llm_chain` 的输入变量，决定是否需要自动设置 `document_variable_name`。

### `input_keys` 属性

In [ ]:
@property
def input_keys(self) -> List[str]:
    extra_keys = [k for k in self.llm_chain.input_keys if k != self.document_variable_name]
    return super().input_keys + extra_keys

- `input_keys` 返回这个链所需的所有输入键。除了 `super().input_keys` 返回的输入键，还包括 `llm_chain` 中不包含在 `document_variable_name` 中的额外键。

### `_get_inputs` 方法

In [ ]:
def _get_inputs(self, docs: List[Document], **kwargs: Any) -> dict:

- `_get_inputs` 方法负责将文档列表格式化并拼接成一个字符串，然后将其与其他 `kwargs` 一起组织成 `llm_chain` 需要的输入字典。
- 该方法先将每个文档通过 `document_prompt` 进行格式化，然后使用 `document_separator` 拼接这些字符串。

### `prompt_length` 方法

In [ ]:
def prompt_length(self, docs: List[Document], **kwargs: Any) -> Optional[int]:

- `prompt_length` 方法计算格式化后的文档列表长度，以便调用者能提前判断这些文档是否会超出 `LLM` 的输入限制。

### combine_docs 和 acombine_docs 方法

In [ ]:
def combine_docs(self, docs: List[Document], callbacks: Callbacks = None, **kwargs: Any) -> Tuple[str, dict]:

async def acombine_docs(
        self, docs: List[Document], callbacks: Callbacks = None, **kwargs: Any
    ) -> Tuple[str, dict]:

- `combine_docs` 是将所有文档合并为一个字符串，并传递给 `LLM` 的主要方法。
- 它调用 `llm_chain.predict` 方法，传递格式化后的文档字符串和其他输入参数。
- `acombine_docs` 是其异步版本。

### `_chain_type` 属性

In [ ]:
@property
def _chain_type(self) -> str:
    return "stuff_documents_chain"

- `_chain_type` 返回链的类型标识符，这里它标识为 `"stuff_documents_chain"`。

### 使用场景
你可以将这个类用于以下场景：

- 当你有一组相关的文档，需要将它们组合成一个`大的`上下文，并通过 LLM 生成总结、回答问题等。
- 它可以在需要将`大量数据传`递给模型的情况下使用。
然而，由于这个类被标记为过时，推荐使用新的构造器 `create_stuff_documents_chain` 来实现类似功能

## demo

In [2]:
from langchain_community.chat_models import ChatTongyi
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
import os

In [3]:

# 从环境变量中获取API密钥，用于初始化 ChatTongyi 模型
api_key = os.getenv("KEY_TONGYI")
if not api_key:
    raise ValueError("API Key is not set. Please ensure that the 'KEY_TONGYI' environment variable is set.")


# 打印并设置Tika服务器的路径，使用本地运行的 Tika Server 解析文件
print(os.path.abspath('./template/tika-server-standard-2.6.0.jar'))
os.environ['TIKA_SERVER_JAR'] = 'http://localhost:9998/'


# 初始化 ChatTongyi 模型，设置文本生成的温度参数，温度越低生成的文本越接近输入
llm = ChatTongyi(
    dashscope_api_key=api_key,
    temperature=0,  # 设置生成文本的倾向，值越小生成的文本越接近输入
    streaming=True
)

e:\AAAAWork\python\LLM_RAG\chain\四种文档预制链\template\tika-server-standard-2.6.0.jar


In [4]:
# 定义模板
document_prompt = PromptTemplate(
  input_variables=['page_content'],
  template="{page_content}"
)

In [5]:
# 定义文档组合后将其传递给 LLM 的变量名称
document_variable_name = "context"

In [6]:
# 定义要使用的 LLM 提示模板，包含组合后的文档
prompt = PromptTemplate.from_template(
    "Summarize this content: {context}"
)

In [7]:
# 将 LLM 和提示模板结合成一个链
llm_chain = LLMChain(llm=llm, prompt=prompt)

e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [8]:
# 初始化 StuffDocumentsChain
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)

e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain/
  warn_deprecated(


In [9]:
# 准备一组文档
docs = [
    Document(page_content="LangChain 是一个强大的库，帮助你轻松处理文档。"),
    Document(page_content="OpenAI 的 GPT-3 模型提供了强大的文本生成能力。"),
    Document(page_content="通过组合这些工具，你可以实现强大的 NLP 应用。")
]

In [ ]:
# 使用 StuffDocumentsChain 处理文档并生成总结
output, _ = chain.combine_docs(docs)

# 打印生成的总结
print(output)

LangChain 是一个强大的库，专为简化文档处理而设计，使得构建高效、可扩展的文本应用变得更加容易。与此同时，OpenAI 的 GPT-3 模型以其卓越的文本生成能力著称，能够根据输入生成高度相关且上下文连贯的文本内容。将 LangChain 与 GPT-3 结合使用，可以显著提升自然语言处理（NLP）应用的性能和复杂度。这种集成不仅能够增强文本理解、生成和处理的能力，还能够应用于各种场景，如智能客服、文本摘要、对话系统等，从而创造出更为智能、交互性更强的应用程序。通过充分利用这两个工具的优势，开发者能够快速构建出具有深度学习能力的 NLP 应用，满足从基础文本分析到高度个性化响应的广泛需求。


In [10]:
input_data = chain._get_inputs(docs=docs)

In [11]:
input_data # 简单拼接

{'context': 'LangChain 是一个强大的库，帮助你轻松处理文档。\n\nOpenAI 的 GPT-3 模型提供了强大的文本生成能力。\n\n通过组合这些工具，你可以实现强大的 NLP 应用。'}